In [1]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

In [1]:
from langchain_community.llms import Ollama

In [2]:
model = Ollama(model='llama3')

In [3]:
model.invoke("Tell me about kubernetes")

'Kubernetes (also known as K8s) is an open-source container orchestration system for automating the deployment, scaling, and management of containerized applications. It was originally designed by Google, and is now maintained by the Cloud Native Computing Foundation (CNCF).\n\n**Key Concepts:**\n\n1. **Containers**: Kubernetes manages containers, which are lightweight and portable packages of software that include everything an application needs to run.\n2. **Pods**: A pod is a logical host for one or more containers, and represents a single instance of deployment. Pods can be used to group multiple containers together.\n3. **Services**: Services provide load balancing, DNS, and network policies for pods.\n4. **Deployments**: Deployments are the way to manage the rollout of new versions of your application. You can have multiple deployments running at the same time, and Kubernetes will automatically roll out updates.\n5. **ReplicaSets**: ReplicaSets ensure that a specified number of r

In [77]:
template="""<|start_header_id|>system<|end_header_id|>
You are an expert in color recognition
<|eot_id|><|start_header_id|>user<|end_header_id|>
return ONLY a color of what is described in message from list:
{colors}
 
If you don't know tell so

USER MESSAGE:\n\n{user_message}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

"""

In [52]:
prompt = PromptTemplate(
    template=template,
    input_variables=["user_message", "colors"]
)

In [24]:
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough

In [53]:
colors = "blye\ngrey\nyellow\nred\npink"

In [54]:
chain = (
    {
        "user_message": RunnablePassthrough(),
        "colors": lambda _: colors
    }
    | prompt
    | model
    | StrOutputParser()
)

In [61]:
chain.invoke('ground')

'I\'m not familiar with the color "ground". It doesn\'t seem to be a typical color name. Can you please provide more context or clarify which color you are referring to?'

In [78]:
few_shot_template = FewShotPromptTemplate(
    example_prompt=PromptTemplate(
        template=template+"{answer}",
        input_variables=["user_message", "colors", "answer"]
    ),
    examples=[
        {
            "user_message": "ground",
            "colors": colors,
            "answer": "I don't know"
        },
        {
            "user_message": "sun",
            "colors": colors,
            "answer": "yellow"
        },
        {
            "user_message": "blood",
            "colors": colors,
            "answer": "red"
        },
    ],
    suffix=template,
    input_variables=["user_message", "colors"]
)

In [79]:
few_shots_chain = (
    {
        "user_message": RunnablePassthrough(),
        "colors": lambda _: colors
    }
    | few_shot_template
    | model
    | StrOutputParser()
)

In [99]:
few_shots_chain.invoke('leaves')

'brown (Note: Since brown isn\'t in the original list, I should have said "I don\'t know" instead. Let me correct that!)\n\nI don\'t know'

In [14]:
tokens_list = """Tether USD (USDT)
BNB (BNB)
Lido Staked Ether (stETH)
USDC (USDC)
Toncoin (TON)
Shiba Inu (SHIB)
Wrapped liquid staked Ether 2.0 (wstETH)
Wrapped Ether (WETH)
Wrapped BTC (WBTC)
Chainlink (LINK)
Uniswap (UNI)
Polygon (MATIC)
PEPE (PEPE)
Wrapped eETH (weETH)
Dai (DAI)
Render Token (RNDR)
Fetch.AI (FET)
NEAR Protocol (NEAR)
Renzo Restaked ETH (ezETH)
Immutable X (IMX)
USDe (USDE)
First Digital USD (FDUSD)
Mantle (MNT)
Injective Protocol (INJ)
OKB (OKB)
ONDO (ONDO)
Cosmos (ATOM)
The Graph (GRT)
Ethereum (ETH)"""

In [37]:

from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

BEGIN_OF_TEXT = "<|begin_of_text|>"
SYSTEM_MESSAGE_START = "<|start_header_id|>system<|end_header_id|>"
USER_MESSAGE_START = "<|start_header_id|>system<|end_header_id|>"
ASSISTANT_MESSAGE_START = "<|start_header_id|>assistant<|end_header_id|>"
END_OF_MESSAGE = "<|eot_id|>"
END_OF_TEXT = "<|end_of_text|>"


system_message_template = """
You are an expert in recognizing what token user wants to transfer.

Here is a list of accepted tokens in format "name (symbol)"
{{ tokens_list }}

Analyze user message from USER_MESSAGE and return pure JSON with three keywords: 'symbol', 'amount' and 'receiver'
'symbol' - is symbol from list.
'amount' - is amount of tokens to transfer. Set to null if not defined.
'receiver' - is a wallet to receive tokens. Set to null if not defined.
"""

def get_prefix_template(system_message: str):
    return f"{BEGIN_OF_TEXT}{SYSTEM_MESSAGE_START}{system_message}{END_OF_MESSAGE}"

prefix = get_prefix_template(system_message=system_message_template)

default_user_message_template = """
USER_MESSAGE:
{{ user_message }}
"""

def get_few_shot_template():
    assistant_message = "{{ assistant_message }}"
    return f"{USER_MESSAGE_START}{default_user_message_template}{END_OF_MESSAGE}{ASSISTANT_MESSAGE_START}{assistant_message}{END_OF_MESSAGE}"


few_shot_template = FewShotPromptTemplate(
    prefix=prefix,
    template_format="jinja2",
    example_prompt=PromptTemplate(
        template=get_few_shot_template(),
        input_variables=["user_message", "assistant_message"],
        template_format="mustache"
    ),
    examples=
    [
        # {
        #     "user_message": "I want to send some USDT",
        #     "assistant_message": "fuck off"
        # },
        {
            "user_message": "I want to send some USDT",
            "assistant_message": "{ 'symbol': 'ETH', 'amount': null, 'receiver': null }"
        },
        # {
        #     "user_message": "I want to send some Ether to 0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97",
        #     "assistant_message": """{ "symbol": "ETH", "amount": null, "receiver": "0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97" }"""
        # },
        # {   
        #     "user_message": "I want to send 10 USDC to 0x302166D919016E9927b2610dB95C756343B93623",
        #     "assistant_message": """{ "symbol": "USDC", "amount": 10, "receiver": "0x302166D919016E9927b2610dB95C756343B93623" }"""
        # }
    ],
    suffix="<|start_header_id|>assistant<|end_header_id|>",
    input_variables=["tokens_list", "user_message"]
)

test_chain = ({
        "tokens_list": lambda _: tokens_list,
        "user_message": RunnablePassthrough()
    }
    | few_shot_template
    # | StrOutputParser()
)

print(test_chain.invoke('I want to send some ether').text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert in recognizing what token user wants to transfer.

Here is a list of accepted tokens in format "name (symbol)"
Tether USD (USDT)
BNB (BNB)
Lido Staked Ether (stETH)
USDC (USDC)
Toncoin (TON)
Shiba Inu (SHIB)
Wrapped liquid staked Ether 2.0 (wstETH)
Wrapped Ether (WETH)
Wrapped BTC (WBTC)
Chainlink (LINK)
Uniswap (UNI)
Polygon (MATIC)
PEPE (PEPE)
Wrapped eETH (weETH)
Dai (DAI)
Render Token (RNDR)
Fetch.AI (FET)
NEAR Protocol (NEAR)
Renzo Restaked ETH (ezETH)
Immutable X (IMX)
USDe (USDE)
First Digital USD (FDUSD)
Mantle (MNT)
Injective Protocol (INJ)
OKB (OKB)
ONDO (ONDO)
Cosmos (ATOM)
The Graph (GRT)
Ethereum (ETH)

Analyze user message from USER_MESSAGE and return pure JSON with three keywords: 'symbol', 'amount' and 'receiver'
'symbol' - is symbol from list.
'amount' - is amount of tokens to transfer. Set to null if not defined.
'receiver' - is a wallet to receive tokens. Set to null if not defined.
<|eot_i